In [1]:
import sys
import glob
import cv2
from keras.models import load_model
import numpy as np
import pickle
from utils.datasets import get_labels
from utils.inference import load_image
from utils.preprocessor import preprocess_input
emotion_model_path = '../trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5'
emotion_labels = get_labels('fer2013')
font = cv2.FONT_HERSHEY_SIMPLEX
# hyper-parameters for bounding boxes shape
emotion_offsets = (20, 40)
emotion_offsets = (0, 0)

Using TensorFlow backend.


In [4]:
emotion_classifier, emotion_target_size=l_model('prob')
emotion_classifier

NameError: name 'l_model' is not defined

Load model

In [29]:
def l_model(layer_label):
    # loading modelsf
    filepath='../../FacialExpressionRecognition/MPM5.hdf5'
    emotion_classifier = load_model(emotion_model_path, compile=False)
    emotion_target_size = emotion_classifier.input_shape[1:3]
    # getting input model shapes for inference
    
#     emotion_classifier.summary()
    if 'fer' in layer_label:
        emotion_classifier=model_generate()
        emotion_classifier.load_weights(filepath)
        emotion_target_size=(48,48)
    if 'emb' in layer_label:
        emotion_classifier.layers.pop()
        emotion_classifier.layers.pop()
        emotion_classifier.outputs = [emotion_classifier.layers[-1].output]
        emotion_classifier.layers[-1].outbound_nodes = []
    
    return emotion_classifier, emotion_target_size
    

Process list of faces

In [ ]:
path_train_negatives = '../imgs/train_crops/positive/*.bmp'
negative_faces = glob.glob(path_train_negatives)
fully = False
i=0
j=0
for image_path in negative_faces:
    try:
        gray_image = load_image(image_path, grayscale=True)
    except:
        j=j+1
        continue
    gray_image = np.squeeze(gray_image)
    gray_image = gray_image.astype('uint8')
    
    gray_face = gray_image
    
    gray_face = cv2.resize(gray_face, (emotion_target_size))
    
    try:
        gray_face = cv2.resize(gray_face, (emotion_target_size))
    except:
        print('exception ignored')
        
    gray_face = preprocess_input(gray_face, True)
    gray_face = np.expand_dims(gray_face, 0)
    gray_face = np.expand_dims(gray_face, -1)
    
    
    i=i+1
print(j,' errors in', len(negative_faces),'faces')

In [3]:
from __future__ import print_function

import cv2
import PIL
from PIL import Image
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adadelta
from keras.utils import np_utils
import numpy as np
import _pickle as cPickle
import numpy
import cv2
import scipy
import csv
from keras import backend as K
K.set_image_dim_ordering('th')
def model_generate():
	filepath='../../FacialExpressionRecognition/MPM5.hdf5'
	img_rows, img_cols = 48, 48
	model = Sequential()
	model.add(Convolution2D(64, 5, 5, border_mode='valid',
							input_shape=(1, img_rows, img_cols)))
	model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
	model.add(keras.layers.convolutional.ZeroPadding2D(padding=(2, 2), dim_ordering='th'))
	model.add(MaxPooling2D(pool_size=(5, 5),strides=(2, 2)))
	  
	model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='th')) 
	model.add(Convolution2D(64, 3, 3))
	model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
	model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='th')) 
	model.add(Convolution2D(64, 3, 3))
	model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
	model.add(keras.layers.convolutional.AveragePooling2D(pool_size=(3, 3),strides=(2, 2)))
	 
	model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='th'))
	model.add(Convolution2D(128, 3, 3))
	model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
	model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='th'))
	model.add(Convolution2D(128, 3, 3))
	model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))

	model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='th'))
	model.add(keras.layers.convolutional.AveragePooling2D(pool_size=(3, 3),strides=(2, 2)))
	 
	model.add(Flatten())
	model.add(Dense(1024))
	model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
	model.add(Dropout(0.2))
	model.add(Dense(1024))
	model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
	model.add(Dropout(0.2))
	 
	model.add(Dense(7))
	model.add(Activation('softmax'))

	ada = Adadelta(lr=0.1, rho=0.95, epsilon=1e-08)
	model.compile(loss='categorical_crossentropy',
				  optimizer=ada,
				  metrics=['accuracy'])
# 	model.load_weights(filepath)
	return model



In [4]:
emotion_classifier, emotion_target_size = l_model('fer_emb')
emotion_classifier.summary()

/home/alexander/.local/lib/python3.5/site-packages/keras/models.py:288: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '
/home/alexander/.local/lib/python3.5/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), padding="valid", input_shape=(1, 48, 48...)`
/home/alexander/.local/lib/python3.5/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `PReLU` call to the Keras 2 API: `PReLU(weights=None, alpha_initializer="zero")`
/home/alexander/.local/lib/python3.5/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `ZeroPadding2D` call to the Keras 2 API: `ZeroPadding2D(padding=(2, 2), data_format="channels_first")`
/home/alexander/.local/lib/python3.5/site-packages/ipykernel_launcher.py:30: UserWarning: Update your `ZeroPadding2D` call to the Keras 2 API: `ZeroPadd

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 44, 44)        1664      
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 64, 44, 44)        123904    
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 64, 48, 48)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 22, 22)        0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 24, 24)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 22, 22)        36928     
_________________________________________________________________
p_re_lu_2 (PReLU)            (None, 64, 22, 22)        30976     
__________

In [24]:
emotion_target_size

(64, 64)

Delete last layer

In [8]:
def load_model_emb():
    emotion_model_path = '../trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5'
    emotion_labels = get_labels('fer2013')
    font = cv2.FONT_HERSHEY_SIMPLEX

    # hyper-parameters for bounding boxes shape
    emotion_offsets = (20, 40)
    emotion_offsets = (0, 0)

    # loading modelsf
    emotion_classifier = load_model(emotion_model_path)

    # getting input model shapes for inference
    emotion_target_size = emotion_classifier.input_shape[1:3]
    emotion_classifier.layers.pop()
    emotion_classifier.layers.pop()
    emotion_classifier.outputs = [emotion_classifier.layers[-1].output]
    emotion_classifier.layers[-1].outbound_nodes = []
    emotion_classifier.summary()
    return emotion_classifier, emotion_target_size

In [13]:
cl=load_model_emb()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 62, 62, 8)    72          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 62, 62, 8)    32          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 62, 62, 8)    0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

/home/alexander/.local/lib/python3.5/site-packages/keras/models.py:288: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


Process faces (pre-last layer)

In [27]:
def get_dict_faces(input_path, layer_label):
    emotion_classifier,emotion_target_size = l_model(layer_label)
    path_train_negatives = input_path
    negative_faces = glob.glob(path_train_negatives)
    fully = False
    i=0
    j=0
    dict_faces={}
    
    for image_path in negative_faces:
        percentage=i/len(negative_faces)*100
        try:
            gray_image = load_image(image_path, grayscale=True)
        except:
            j=j+1
            continue
        gray_image = np.squeeze(gray_image)
        gray_image = gray_image.astype('uint8')

        gray_face = gray_image

        gray_face = cv2.resize(gray_face, (emotion_target_size))

        try:
            gray_face = cv2.resize(gray_face, (emotion_target_size))
        except:
            print('exception ignored')

        gray_face = preprocess_input(gray_face, True)
        gray_face = np.expand_dims(gray_face, 0)
        if 'fer' in layer_label:
            gray_face = np.expand_dims(gray_face, 0)
        if 'fer' not in layer_label:
            gray_face = np.expand_dims(gray_face, -1)
        dict_faces[str(image_path)]=emotion_classifier.predict(gray_face)
    #     print(dict_faces)
    #     with open('negative.txt', 'a') as file:
    #         file.write(str(image_path) + ' ')
    #         for prob in emotion_classifier.predict(gray_face)[0]:
    #             file.write( str(prob) + ' ')
    #         file.write('\n')
        i=i+1
        sys.stdout.write("\r%d%%" % percentage)
        sys.stdout.flush()
    #     print(i)
    print(j,' errors in', len(negative_faces),'faces')
    return dict_faces

In [25]:
def save_dict_faces_to_file(em_label, layer_label, val_or_train):
    f_name=em_label+'_'+layer_label+'_'+val_or_train
    file=open(f_name, 'wb')
    pickle.dump(get_dict_faces('../imgs/'+val_or_train+'_crops/'+em_label+'/*', layer_label),file)

In [11]:
def save_all_dict_faces(layer_label):
    val_or_train = 'train'
    save_dict_faces_to_file('negative', layer_label, val_or_train)
    save_dict_faces_to_file('positive', layer_label, val_or_train)
    save_dict_faces_to_file('neutral', layer_label, val_or_train)
    val_or_train = 'val'
    save_dict_faces_to_file('negative', layer_label, val_or_train)
    save_dict_faces_to_file('positive', layer_label, val_or_train)
    save_dict_faces_to_file('neutral', layer_label, val_or_train)

In [ ]:
save_all_dict_faces(layer_label='fer_emb')

/home/alexander/.local/lib/python3.5/site-packages/keras/models.py:288: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '
/home/alexander/.local/lib/python3.5/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), input_shape=(1, 48, 48..., padding="valid")`
/home/alexander/.local/lib/python3.5/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `PReLU` call to the Keras 2 API: `PReLU(alpha_initializer="zero", weights=None)`
/home/alexander/.local/lib/python3.5/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `ZeroPadding2D` call to the Keras 2 API: `ZeroPadding2D(data_format="channels_first", padding=(2, 2))`
/home/alexander/.local/lib/python3.5/site-packages/ipykernel_launcher.py:30: UserWarning: Update your `ZeroPadding2D` call to the Keras 2 API: `ZeroPadd

99%139  errors in 15401 faces
99%64  errors in 9400 faces
99%70  errors in 10046 faces
99%69  errors in 8197 faces
99%48  errors in 4995 faces
86%

In [19]:
file=open('negative_pre_last_val.txt', 'wb')
pickle.dump(dict_faces,file)

In [14]:
emotion_classifier, emotion_target_size = l_model('prob')
gray_image = load_image('../imgs/train_crops/negative/TrainNegative000045d4-642.jpgface0.bmp', grayscale=True)
gray_image = np.squeeze(gray_image)
gray_image = gray_image.astype('uint8')
emotion_target_size=(64,64)
print(emotion_target_size)
gray_face = gray_image
print(gray_face.shape)
gray_face = cv2.resize(gray_face, (emotion_target_size))
print(gray_face.shape)
try:
    gray_face = cv2.resize(gray_face, (emotion_target_size))
except:
    print('exception ignored')

gray_face = preprocess_input(gray_face, True)
gray_face = np.expand_dims(gray_face, 0)
gray_face = np.expand_dims(gray_face, -1)
# gray_face = gray_face[None,:]
gray_face.shape
emotion_classifier.predict(gray_face)

/home/alexander/.local/lib/python3.5/site-packages/keras/models.py:288: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


(64, 64)
(107, 87)
(64, 64)


array([[  1.96372315e-01,   7.60058492e-06,   2.60079920e-01,
          5.83580477e-05,   4.94902551e-01,   1.58757064e-02,
          3.27035487e-02]], dtype=float32)

In [11]:
emotion_classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 64, 44, 44)        1664      
_________________________________________________________________
p_re_lu_22 (PReLU)           (None, 64, 44, 44)        123904    
_________________________________________________________________
zero_padding2d_19 (ZeroPaddi (None, 64, 48, 48)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 64, 22, 22)        0         
_________________________________________________________________
zero_padding2d_20 (ZeroPaddi (None, 64, 24, 24)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 64, 22, 22)        36928     
_________________________________________________________________
p_re_lu_23 (PReLU)           (None, 64, 22, 22)        30976     
__________

In [31]:

save_dict_faces_to_file('negative', 'emb', 'train')



99%139  errors in 15401 faces


In [20]:
emotion_classifier, emotion_target_size = l_model('prob')
emotion_classifier.load_weights('../trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5')
gray_image = load_image('../imgs/train_crops/negative/TrainNegative000045d4-642.jpgface0.bmp', grayscale=True)
gray_image = np.squeeze(gray_image)
gray_image = gray_image.astype('uint8')
emotion_target_size=(64,64)
print(emotion_target_size)
gray_face = gray_image
print(gray_face.shape)
gray_face = cv2.resize(gray_face, (emotion_target_size))
print(gray_face.shape)
try:
    gray_face = cv2.resize(gray_face, (emotion_target_size))
except:
    print('exception ignored')

gray_face = preprocess_input(gray_face, True)
gray_face = np.expand_dims(gray_face, 0)
gray_face = np.expand_dims(gray_face, -1)
emotion_classifier.predict(gray_face)

/home/alexander/.local/lib/python3.5/site-packages/keras/models.py:288: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


(64, 64)
(107, 87)
(64, 64)


array([[  1.96372315e-01,   7.60058492e-06,   2.60079920e-01,
          5.83580477e-05,   4.94902551e-01,   1.58757064e-02,
          3.27035487e-02]], dtype=float32)

In [30]:
emotion_classifier, emotion_target_size = l_model('prob')
# emotion_classifier.load_weights('../trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5')
gray_image = load_image('../imgs/train_crops/negative/TrainNegative000045d4-642.jpgface0.bmp', grayscale=True)
gray_image = np.squeeze(gray_image)
gray_image = gray_image.astype('uint8')
emotion_target_size=(64,64)
print(emotion_target_size)
gray_face = gray_image
print(gray_face.shape)
gray_face = cv2.resize(gray_face, (emotion_target_size))
print(gray_face.shape)
try:
    gray_face = cv2.resize(gray_face, (emotion_target_size))
except:
    print('exception ignored')

gray_face = preprocess_input(gray_face, True)
gray_face = np.expand_dims(gray_face, 0)
gray_face = np.expand_dims(gray_face, -1)
emotion_classifier.predict(gray_face)

(64, 64)
(107, 87)
(64, 64)


array([[  1.96372315e-01,   7.60058492e-06,   2.60079920e-01,
          5.83580477e-05,   4.94902551e-01,   1.58757064e-02,
          3.27035487e-02]], dtype=float32)

In [22]:
emotion_classifier.load_weights('../trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5')
